In [1]:
import json
import pandas as pd
import numpy as np

import requests

from tqdm import tqdm_notebook

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
# import SVD
# import accuracy

import warnings; warnings.simplefilter('ignore')

# 성인 들이 읽는 책을 책소개를 통해 비슷한 책 추천해주기

In [4]:
# 데이터 불러오기
adult = pd.read_csv('./data/final/content_re/adult_df.csv',encoding='utf-8')

In [5]:
adult.head(3)

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,loan_count,bookImageURL
0,1,1,여행의 이유 :김영하 산문,지은이: 김영하,문학동네,2019.0,9788954655972,3810.0,NaN,816.70,"4,899",https://image.aladin.co.kr/product/18827/60/co...
1,2,2,천년의 질문 :조정래 장편소설,저자: 조정래,해냄,2019.0,9788965746829,4810.0,1.0,813.62,"4,197",https://image.aladin.co.kr/product/19386/11/co...
2,3,3,선량한 차별주의자,김지혜 지음,창비,2019.0,9788936477196,3300.0,NaN,334.00,"4,140",https://image.aladin.co.kr/product/19874/76/co...


In [8]:
len(adult)

995

## merge로 책소개 가져오기

In [7]:
# 책 소개글이 포함되어 있는 전체 데이터 가져오기 
book = pd.read_csv('./data/final/library_book_df.csv',encoding='utf-8')

In [9]:
adult_result=pd.merge(adult,book,on='isbn13',how='left')

In [11]:
len(adult_result)

995

In [12]:
# 데이터프레임으로 바꾸기
adult_result.to_csv('./data/final/content_re/adult_result.csv',encoding='utf-8-sig',index=False)

In [13]:
# 직접 엑셀에서 수정 후 데이터 불러오기
adult_result=pd.read_excel('./data/final/content_re/adult_result.xlsx')

In [14]:
adult_result.head(3)

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,loan_count,bookImageURL,keyword,describe
0,1,1,여행의 이유 :김영하 산문,지은이: 김영하,문학동네,2019.0,9788954655972,3810.0,NaN,816.70,4899,https://image.aladin.co.kr/product/18827/60/co...,"['여행', '김영하', '여행의', '사유', '산문', '인간', '매혹적', ...",“이 책을 쓰는 데 내 모든 여행의 경험이 필요했다”여행-일상-여행의 고리를 잇는 ...
1,2,2,천년의 질문 :조정래 장편소설,저자: 조정래,해냄,2019.0,9788965746829,4810.0,1.0,813.62,4197,https://image.aladin.co.kr/product/19386/11/co...,"['div', 'li', 'ul', 'strong', 'span', 'br', 'b...",폐허의 역사를 극복하고 경쟁의 도시로 거듭나 세계의 주목을 받는 대한민국에서 기업가...
2,3,3,선량한 차별주의자,김지혜 지음,창비,2019.0,9788936477196,3300.0,NaN,334.00,4140,https://image.aladin.co.kr/product/19874/76/co...,"['차별', '선량', '평등', '연구', '현장', '혐오', '사람', '사회...",선량한 차별주의자들의 세상에서 &#x0D;평등을 외치는 당신을 위한 안내서 &#x0...


In [15]:
len(adult_result)

995

## 크롤링을 통해 책 소개 가져오기

In [19]:
# 크롤링
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
from urllib.error import URLError

In [20]:
# 오픈을 할때 예외처리를 통해 만약 오류가 생기면 원인을 알수 있게 함수 있음
try :
    html = urlopen('https://pythondojang.bitbucket.io/weather/observation/currentweather.html')
except HTTPError as he :
    print('http error')
except URLError as ue :
    print('url error')
else :
    soup = BeautifulSoup(html.read(), 'html.parser')

In [21]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
import time

In [22]:
# isbn 리스트 생성
adult_isbn_list = list(adult_result['isbn13'].astype(str))

In [28]:
# 자동화 웹 페이지 열기
path = './driver/chromedriver.exe'
driver = webdriver.Chrome(path)
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="e9cd31463d9cd5485463371fc3a2ef6e")>

In [29]:
isbn_url = 'http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode='
rest_url = '&orderClick=LAG&Kc='

In [30]:
intro = []

for i in tqdm_notebook(range(len(adult_isbn_list))) :
    url = isbn_url + str(adult_isbn_list[i]) + rest_url
    driver.get(url)
    
    a_list = driver.find_elements_by_css_selector('div.box_detail_article')
    time.sleep(1.5)
    a_list = driver.find_elements_by_css_selector('div.box_detail_article')
    
    if len(a_list) > 0 :
        intro.append(a_list[0].text)
    else :
        intro.append('')
        continue

In [ ]:
len(adult_isbn_list)

In [31]:
len(intro)

995

In [33]:
adult_result_revised=adult_result

In [35]:
adult_result_revised['describe']=intro

In [37]:
adult_result_revised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                995 non-null    int64  
 1   ranking           995 non-null    int64  
 2   bookname          995 non-null    object 
 3   authors           995 non-null    object 
 4   publisher         988 non-null    object 
 5   publication_year  970 non-null    float64
 6   isbn13            995 non-null    int64  
 7   addition_symbol   977 non-null    float64
 8   vol               181 non-null    float64
 9   class_no          983 non-null    float64
 10  loan_count        995 non-null    int64  
 11  bookImageURL      987 non-null    object 
 12  keyword           848 non-null    object 
 13  describe          995 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 109.0+ KB


In [44]:
adult_result_revised.to_csv('./data/final/content_re/adult_result_revised.csv',encoding='utf-8-sig',index=False)

In [42]:
adult_result_revised['describe'][0].index

<function str.index>

In [45]:
adult_result_revised=pd.read_excel('./data/final/content_re/adult_result_revised.xlsx')

In [48]:
adult_result_revised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                995 non-null    int64  
 1   ranking           995 non-null    int64  
 2   bookname          995 non-null    object 
 3   authors           995 non-null    object 
 4   publisher         988 non-null    object 
 5   publication_year  970 non-null    float64
 6   isbn13            995 non-null    int64  
 7   addition_symbol   977 non-null    float64
 8   vol               181 non-null    float64
 9   class_no          983 non-null    float64
 10  loan_count        995 non-null    int64  
 11  bookImageURL      987 non-null    object 
 12  keyword           848 non-null    object 
 13  describe          983 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 109.0+ KB


In [49]:
# 책소개가 없는 데이터만 따로 뽑기
adult_na_df=adult_result_revised[adult_result_revised['describe'].isnull()]

In [56]:
len(adult_na_df)

12

In [55]:
# 책소개가 있는 데이터만 따로 뽑기
adult_revised=adult_result_revised[pd.notnull(adult_result_revised['describe'])]

In [57]:
len(adult_revised)

983

In [51]:
adult_na_df.to_csv('./data/final/content_re/adult_na_df.csv',encoding='utf-8-sig',index=False)

In [53]:
# 책소개 이미지 없는 책들을 직접 excel에 찾아서 넣기
adult_na_df=pd.read_excel('./data/final/content_re/adult_na_df.xlsx')

In [54]:
adult_na_df

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,loan_count,bookImageURL,keyword,describe
0,116,116,존리의 부자되기 습관 :대한민국 경제독립 액션 플랜,존리 지음,지식노마드,2020.0,9791187481720,13320.0,NaN,327.04,1371,https://image.aladin.co.kr/product/22631/95/co...,"['펀드', '경제독립', '투자', '금융문맹', '부자', '시작', '한국',...",돈을 위해 일하지 말고 돈이 당신을 위해 일하게 하라!!\n열심히 일하는데 왜 돈에...
1,469,469,당신이 옳다 :정혜신의 적정심리학,정혜신,NaN,NaN,9788965746669,NaN,NaN,189.00,684,http://image.kyobobook.co.kr/images/book/large...,"['공감', '심리', '치유', '심리치유', '콤플렉스', '내공']",안정적인 일상을 위해 스스로 마음의 문제를 해결할 수 있도록 도와주는 공감 행동지침...
2,541,541,82년생 김지영 :조남주 장편소설,조남주 지음,NaN,2018.0,9788937473135,3810.0,NaN,813.70,631,http://image.kyobobook.co.kr/images/book/large...,"['페미니즘', '인생', '한국 여성', '성 차별', '고백', '성 평등']","공포, 피로, 당황, 놀람, 혼란, 좌절의 연속에 대한 한국 여자의 인생 현장 보고..."
3,648,648,남아 있는 시간을 위하여:100세 철학자의 대표산문선,김형석,NaN,NaN,9788934980629,NaN,NaN,104.00,565,http://image.kyobobook.co.kr/images/book/large...,NaN,인생의 마지막에 가까워질수록 더욱 간절해지는 선하고 아름다운 삶을 향한 고민의 소산...
4,658,658,방구석 미술관:가볍고 편하게 시작하는 유쾌한 교양 미술,조원재 지음;위키피디아 본문사진,NaN,NaN,9788968331862,NaN,NaN,650.40,558,http://image.kyobobook.co.kr/images/book/large...,"['미술이야기', '미술', '화가', '미술사', '오르세', '세잔']",조원재 작가와 함께 즐기는 ‘프라이빗 미술관 에디션’ _ 오르세 미술관 편\n『방구...
5,663,660,인생 우화,류시화,NaN,NaN,9791186686348,NaN,NaN,813.80,556,http://image.kyobobook.co.kr/images/book/large...,NaN,우리 안의 바보는 어떤 엉뚱한 방식으로 문제를 해결하며 살아갈까?\n류시화 시인이 ...
6,699,698,인생 수업,"엘리자베스 퀴블러 로스,류시화 옮김",이레,2006.0,9788957090817,3840.0,NaN,199.10,535,https://bookthumb-phinf.pstatic.net/cover/024/...,"['자신', '마지막', '바쇼하이쿠선집', '삶으로다시떠오르기', '사랑하라한번도...",배움을 얻는다는 것은 다른 사람이 아닌 자기 자신의 인생을 사는 것을 의미합니다. ...
7,701,701,옛날에 내가 죽은 집,지은이: 히가시노 게이고 ;옮긴이: 최고은,김영사,2019.0,9788934995289,3830.0,NaN,833.60,533,https://image.aladin.co.kr/product/19711/55/co...,"['일본', '옛날에내가죽은집', '히가시노게이고', '오사카소년탐정단', '나미야...","극한의 제약, 무한의 상상력,\n히가시노 게이고만이 가능한 고밀도 미스터리!\n\n..."
8,738,733,(법륜 스님의) 행복 :행복해지고 싶지만 길을 몰라 헤매는 당신에게,법륜 지음 ;최승미 그림,나무의마음,2017.0,9788954637169,3810.0,NaN,199.10,524,http://image.kyobobook.co.kr/images/book/large...,NaN,당신은 행복해질 권리가 있다\n법륜 스님은 지난 30년 동안 우리나라 전국방방곡곡은...
9,748,748,(있는 그대로) 참 소중한 너라서 :지금도 충분히 아름다운 그대에게,김지훈 지음,RHK(알에이치코리아),2016.0,9791196484224,3810.0,NaN,818.00,519,https://bookthumb-phinf.pstatic.net/cover/107/...,"['사랑', '독자', '행복', '고민', '마음', '위로', '공감', '감사...",있는 그대로 참 소중한 당신에게 전하는 진심 가득한 위로의 말\n2년이 넘도록 전국...


In [58]:
# 최종 adult 데이터 넣기
adult_final_df=pd.concat([adult_na_df,adult_revised],ignore_index=True)

- 최종 데이터 저장

In [ ]:
# 책 제목 중복데이터 제거 
adult_final_df=adult_final_df.drop_duplicates(['bookname'], keep='first')

In [68]:
len(adult_final_df)

891

In [ ]:
adult_final_df.reset_index(drop=True,inplace=True)

In [67]:
adult_final_df.to_csv('./data/final/content_re/adult_final_df.csv',encoding='utf-8-sig',index=False)

In [60]:
adult_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                995 non-null    int64  
 1   ranking           995 non-null    int64  
 2   bookname          995 non-null    object 
 3   authors           995 non-null    object 
 4   publisher         988 non-null    object 
 5   publication_year  970 non-null    float64
 6   isbn13            995 non-null    int64  
 7   addition_symbol   977 non-null    float64
 8   vol               181 non-null    float64
 9   class_no          983 non-null    float64
 10  loan_count        995 non-null    int64  
 11  bookImageURL      995 non-null    object 
 12  keyword           848 non-null    object 
 13  describe          995 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 109.0+ KB


## TF-IDF를 통한 어른 책 내용 기반 추천

In [102]:
adult_final_df=pd.read_excel('./data/final/content_re/adult_final_df.xlsx')

In [104]:
adult_final_df

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,loan_count,bookImageURL,keyword,describe
0,116,116,존리의 부자되기 습관 :대한민국 경제독립 액션 플랜,존리 지음,지식노마드,2020.0,9791187481720,13320.0,NaN,327.04,1371,https://image.aladin.co.kr/product/22631/95/co...,"['펀드', '경제독립', '투자', '금융문맹', '부자', '시작', '한국',...",돈을 위해 일하지 말고 돈이 당신을 위해 일하게 하라!!\n열심히 일하는데 왜 돈에...
1,469,469,당신이 옳다 :정혜신의 적정심리학,정혜신,NaN,NaN,9788965746669,NaN,NaN,189.00,684,http://image.kyobobook.co.kr/images/book/large...,"['공감', '심리', '치유', '심리치유', '콤플렉스', '내공']",안정적인 일상을 위해 스스로 마음의 문제를 해결할 수 있도록 도와주는 공감 행동지침...
2,648,648,남아 있는 시간을 위하여:100세 철학자의 대표산문선,김형석,NaN,NaN,9788934980629,NaN,NaN,104.00,565,http://image.kyobobook.co.kr/images/book/large...,NaN,인생의 마지막에 가까워질수록 더욱 간절해지는 선하고 아름다운 삶을 향한 고민의 소산...
3,663,660,인생 우화,류시화,NaN,NaN,9791186686348,NaN,NaN,813.80,556,http://image.kyobobook.co.kr/images/book/large...,NaN,우리 안의 바보는 어떤 엉뚱한 방식으로 문제를 해결하며 살아갈까?\n류시화 시인이 ...
4,699,698,인생 수업,"엘리자베스 퀴블러 로스,류시화 옮김",이레,2006.0,9788957090817,3840.0,NaN,199.10,535,https://bookthumb-phinf.pstatic.net/cover/024/...,"['자신', '마지막', '바쇼하이쿠선집', '삶으로다시떠오르기', '사랑하라한번도...",배움을 얻는다는 것은 다른 사람이 아닌 자기 자신의 인생을 사는 것을 의미합니다. ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,995,990,수도권 알짜 부동산 답사기 :알짜 노하우,김학렬 지음,지혜로,2019.0,9791187799108,13320.0,NaN,327.87,445,https://image.aladin.co.kr/product/19916/14/co...,"['서울', '레시피', '동네', '부동산', '지역분석', '인천', '지역',...",대한민국 역사상 가장 강력한 부동산 규제정책이 펼쳐지고 있다. 내 집 마련을 해야 ...
884,996,990,사라진 세계,톰 스웨터리치 지음 ;장호연 옮김,허블,2020.0,9791190090070,3840.0,NaN,843.60,445,https://image.aladin.co.kr/product/23332/89/co...,NaN,"허리우드가 주목하는, 특히, 아카데미 후보작 《디스트릭트9》으로 세계적인 명성을 쌓..."
885,997,990,나를 아프게 하지 않는다 :상처만 주는 가짜 자존감 나를 지키는 진짜 자존감,전미경 지음,지와인,2019.0,9791196533472,3180.0,NaN,189.10,445,https://image.aladin.co.kr/product/21831/77/co...,"['자존감', '사람', '자기', '감정', '가짜', '관계', '시그니처', ...",우리는 어쩌다 자존감의 시대를 살게 되었을까? 원래 자존감이라는 게 이렇게 만들어지...
886,998,998,김미경의 인생미답 :살다 보면 누구나 마주하는 작고 소소한 질문들,김미경 지음,한경BP,2016.0,9788947540926,3320.0,NaN,199.10,444,http://image.aladin.co.kr/product/8193/51/cove...,"['김미경', '자신', '문제', '인생', '사랑', '소소', '질문', '김...",스타 강사 김미경은 오랫동안 데리고 살아온 고민과 나름의 생각을 모두와 공유하고 싶...


In [105]:
adult_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                888 non-null    int64  
 1   ranking           888 non-null    int64  
 2   bookname          888 non-null    object 
 3   authors           888 non-null    object 
 4   publisher         883 non-null    object 
 5   publication_year  875 non-null    float64
 6   isbn13            888 non-null    int64  
 7   addition_symbol   872 non-null    float64
 8   vol               84 non-null     float64
 9   class_no          876 non-null    float64
 10  loan_count        888 non-null    int64  
 11  bookImageURL      888 non-null    object 
 12  keyword           842 non-null    object 
 13  describe          888 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 97.2+ KB


In [106]:
# 블용어 txt파일 리스트 가져오기
sw=pd.read_csv('./data/final/한국어불용어100.txt', sep = "\t",  engine='python', encoding = "utf-8",header=None)

In [107]:
stop_words=list(sw[0].dropna())

In [108]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(adult_final_df['describe'])

In [109]:
tfidf_matrix.shape

(888, 152001)

In [110]:
# linear_kernel는 두 벡터의 dot product 이다.
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [111]:
cosine_sim

array([[1.        , 0.01900273, 0.01129969, ..., 0.00958737, 0.001026  ,
        0.0017967 ],
       [0.01900273, 1.        , 0.02005707, ..., 0.02334484, 0.01152541,
        0.01091262],
       [0.01129969, 0.02005707, 1.        , ..., 0.00772229, 0.00381778,
        0.00446295],
       ...,
       [0.00958737, 0.02334484, 0.00772229, ..., 1.        , 0.00306536,
        0.00486766],
       [0.001026  , 0.01152541, 0.00381778, ..., 0.00306536, 1.        ,
        0.00330135],
       [0.0017967 , 0.01091262, 0.00446295, ..., 0.00486766, 0.00330135,
        1.        ]])

In [112]:
titles = adult_final_df['bookname']
indices = pd.Series(adult_final_df.index, index=adult_final_df['bookname'])

print(titles.head(), indices.head())

0     존리의 부자되기 습관 :대한민국 경제독립 액션 플랜
1               당신이 옳다 :정혜신의 적정심리학
2    남아 있는 시간을 위하여:100세 철학자의 대표산문선
3                            인생 우화
4                            인생 수업
Name: bookname, dtype: object bookname
존리의 부자되기 습관 :대한민국 경제독립 액션 플랜     0
당신이 옳다 :정혜신의 적정심리학               1
남아 있는 시간을 위하여:100세 철학자의 대표산문선    2
인생 우화                            3
인생 수업                            4
dtype: int64


In [113]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [114]:
get_recommendations('존리의 부자되기 습관 :대한민국 경제독립 액션 플랜').head(5)

433    진짜 부자 가짜 부자 =사경인 회계사의 부자 되는 돈 공부 /Real rich po...
532             백만장자 시크릿 :부를 끌어당기는 17가지 원칙을 알려주는 부자 매뉴얼 
287                          부자언니 부자특강 :평범한 월급쟁이 부자되는 공식
629                     부자 아빠 가난한 아빠:부자들이 들려주는 돈과 투자의 비밀
391    부자 되는 법을 가르쳐 드립니다 :죄책감도, 핑계도, 거짓도 없다. 정말로 효과 있...
Name: bookname, dtype: object

In [116]:
get_recommendations('방구석 미술관 :가볍고 편하게 시작하는 유쾌한 교양 미술').head(5)

548                      다락방 미술관 :그림 속 숨어있는 이야기 
838                       (난생 처음 한번 공부하는) 미술 이야기
624    모든 관계는 말투에서 시작된다 :기분 좋은 사람으로 기억되는 사소한 습관 
681            미술관에 간 심리학 :나는 심리학을 공부하러 미술관에 간다 
85         내가 원하는 것을 나도 모를 때 :잃어버린 나를 찾는 인생의 문장들
Name: bookname, dtype: object

In [98]:
adult_final_df['bookname'][11]

'나는 죽을 때까지 재미있게 살고 싶다'

In [117]:
get_recommendations('나는 죽을 때까지 재미있게 살고 싶다').head(5)

702    나이든 나와 살아가는 법 :흔들리지 않고 의연하게 나이 들 수 있는 후반생의 마음 사전 
427                           인생 수업 :잘 물든 단풍은 봄꽃보다 아름답다 
314               나는 내 나이가 참 좋다 :우아하고 지혜롭게 세월의 강을 항해하는 법
289                       (어차피 살 거라면) 백 살까지 유쾌하게 나이 드는 법
504          50 이후, 인생을 결정하는 열 가지 힘 - 하버드 심리학 거장의 마지막 강의
Name: bookname, dtype: object

In [118]:
get_recommendations('아주 작은 습관의 힘 :최고의 변화는 어떻게 만들어지는가').head(5)

312                      나는 습관을 조금 바꾸기로 했다
236     해빗 =내 안의 충동을 이겨내는 습관 설계의 법칙 /Habit
499     백년운동 :척추·관절 아프지 않게 100세까지 운동하는 방법 
227    떨림과 울림 :물리학자 김상욱이 바라본 우주와 세계 그리고 우리
402           부자언니 부자연습 :가난한 공주 부자되기 프로젝트 
Name: bookname, dtype: object